In [ ]:
#@title <b>v-- Enter your model below and then click this to start Koboldcpp</b>

Model = "https://huggingface.co/TheBloke/Airoboros-L2-13B-2.2-GGUF/resolve/main/airoboros-l2-13b-2.2.Q4_K_M.gguf" #@param [""]{allow-input: true}
Layers = 43 #@param [43]{allow-input: true}

%cd /content
!git clone https://github.com/LostRuins/koboldcpp
%cd /content/koboldcpp
!make LLAMA_CUBLAS=1

!wget $Model -O model.ggml
!wget -c https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64
!nohup ./cloudflared-linux-amd64 tunnel --url http://localhost:5001 &
!sleep 10
!cat nohup.out
!python koboldcpp.py model.ggml --usecublas 0 mmq --gpulayers $Layers --hordeconfig concedo
